# $\bm{26}$ $\enspace$ **网络中的网络 NiN**

## $\bm{26.1}$ $\enspace$ **NiN 块**

![NiN](image/NiN.jpg)

In [ ]:
import torch
from torch import nn
from d2l import torch as d2l


def nin_block(in_channels, out_channels, kernel_size, strides, paddings):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, strides, paddings),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1),
        nn.ReLU()
    )

## $\bm{26.2}$ $\enspace$ **NiN 模型**

NiN 和 AlexNet 之间的一个显著区别是 NiN 完全取消了全连接层。相反，NiN 使用一个 NiN 块，其输出通道数等于标签类别的数量。最后放一个全局平均汇聚层 (global average pooling layer)，生成一个对数几率 (logits)。NiN 设计的一个优点是，它显著减少了模型所需参数的数量。

In [ ]:
net = nn.Sequential(
    nin_block(1, 96, kernel_size=11, strides=4, paddings=0),
    nn.MaxPool2d(3, stride=2),
    nin_block(96, 256, kernel_size=5, strides=1, paddings=2),
    nn.MaxPool2d(3, stride=2),
    nin_block(256, 384, kernel_size=3, strides=1, paddings=1),
    nn.MaxPool2d(3, stride=2),
    nn.Dropout(0.5),
    # 标签类别数是10
    nin_block(384, 10, kernel_size=3, strides=1, paddings=1),
    nn.AdaptiveAvgPool2d((1, 1)),
    # 将四维的输出转成二维的输出，其形状变为10
    nn.Flatten()
)

In [ ]:
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__, "output shape:\t", X.shape)

## $\bm{26.3}$ $\enspace$ **训练模型**

In [ ]:
lr, num_epochs, batch_size = 0.1, 10, 128
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())